In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [3]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [4]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [38]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,Borough,Neighborhood,Latitude,Longitude,PostalCode
0,North York\n,Parkwoods\n,43.752935,-79.335641,M3A\n
1,North York\n,Victoria Village\n,43.728102,-79.311890,M4A\n
2,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041,M5A\n
3,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211,M6A\n
4,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.661790,-79.389390,M7A\n
...,...,...,...,...,...
221,Not assigned\n,M1Z\n,43.648690,-79.385440,M9Y\n
222,Not assigned\n,M3Z\n,43.648690,-79.385440,M2Z\n
223,Not assigned\n,M5Z\n,43.648690,-79.385440,M4Z\n
224,Not assigned\n,M7Z\n,43.648690,-79.385440,M6Z\n


In [46]:
neighbors.drop(neighbors[neighbors['Borough']=="Not assigned\n"].index,axis=0, inplace=True)

In [48]:
neighbors = neighbors[~np.isnan(neighbors.Latitude)]
neighbors.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude,PostalCode
0,North York\n,Parkwoods\n,43.752935,-79.335641,M3A\n
1,North York\n,Victoria Village\n,43.728102,-79.311890,M4A\n
2,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041,M5A\n
3,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211,M6A\n
4,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.661790,-79.389390,M7A\n
...,...,...,...,...,...
177,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n",43.653340,-79.509766,M8X\n
178,Downtown Toronto\n,Church and Wellesley\n,43.666659,-79.381472,M4Y\n
179,East Toronto\n,"Business reply mail Processing Centre, South C...",43.648700,-79.385450,M7Y\n
180,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017,M8Y\n


In [50]:
neighbors.head()

,Borough,Neighborhood,Latitude,Longitude,PostalCode
0,North York\n,Parkwoods\n,43.752935,-79.335641,M3A\n
1,North York\n,Victoria Village\n,43.728102,-79.311890,M4A\n
2,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041,M5A\n
3,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211,M6A\n
4,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.661790,-79.389390,M7A\n
